Preprocessing

In [ ]:
#connect to gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks"

 data   data_preparer.py  'H&M.ipynb'   __pycache__   train.ipynb


In [ ]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
import sys
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks")
import data_preparer
from pathlib import Path
from keras.utils import to_categorical
import torch
import tensorflow as tf

In [ ]:
#prepare mask for vectors
rel = data_preparer.get_vectors_relative_position(7, 180, 20)
abss = data_preparer.get_vectors_absolute_position(rel)
mask = data_preparer.get_mask(abss)

Spacing: 30.0 degrees


In [ ]:
#read recon file
file = "sub001_2drt_07_grandfather1_r1_recon.h5"
filepath = f"/content/drive/My Drive/Colab Notebooks/data/sub001/2drt/recon/{file}"
recon = data_preparer.get_recon(filepath)

In [ ]:
#read timestamps file
filepath = "/content/drive/My Drive/Colab Notebooks/data/sub001/timestamps/sub001_2drt_01_vcv1_r1_recon.csv"
timestamps = data_preparer.read_timestamps(filepath)

In [ ]:
#get vectors of the first row of timestamps
vectors = data_preparer.get_pixel_data(recon, abss, timestamps.iloc[0])

In [ ]:
vector1 = data_preparer.get_pixel_data(recon, abss, timestamps.iloc[0])
vector2 = data_preparer.get_pixel_data(recon, abss, timestamps.iloc[1])


In [ ]:
vector1.shape

(7, 20, 52)

In [ ]:
timestamps['total frames'] = timestamps['last_frame'] - timestamps['first_frame'] #create new col to get the higehst number of frames

In [ ]:
highest_framenr = round(timestamps['total frames'].max())
highest_framenr

80

In [ ]:
#convert np array to tensors from pytroch
tensor1 = torch.from_numpy(vector1.copy())
tensor2 = torch.from_numpy(vector2.copy())

In [ ]:
# expand tensor2 along the third dimension to match the size of tensor1
tensor2 = torch.cat((tensor2, torch.zeros(7, 20, 12)), dim=2)
# stack the tensors along a new first dimension
stacked_tensor = torch.stack([tensor1, tensor2], dim=0)

In [ ]:
stacked_tensor.shape

torch.Size([2, 7, 20, 52])

In [ ]:
highest_framenr - round(timestamps.iloc[0]['total frames'])

28

In [ ]:
files = ['sub001-2drt_01_vcv1_r1_recon', 'sub001_2drt_02_vcv2_r1_recon']
X,y = get_data(files)

In [ ]:
#TO DO: write function which can read multiple files an output it to X (pixel vektor arry) and y (lettercat)
def get_data(filename_list):
  #prepare mask for vectors
  rel = data_preparer.get_vectors_relative_position(7, 180, 20)
  abss = data_preparer.get_vectors_absolute_position(rel)
  mask = data_preparer.get_mask(abss)

  #read timestamp files and append them to one large dataframe
  all_timestamps = pd.DataFrame({})
  for file in filename_list:
    filepath = f"/content/drive/My Drive/Colab Notebooks/data/sub001/timestamps/{file}.csv"
    timestamps = data_preparer.read_timestamps(filepath)
    timestamps['recon'] = file
    all_timestamps = all_timestamps.append(timestamps, reset_index=True)

#now get pixel vector data with iterating over the large dataframe
  tensor_list = []
  all_timestamps['total frames'] = all_timestamps['last_frame'] - all_timestamps['first_frame'] #create new col to get the higehst number of frames
  highest_framenr = round(all_timestamps['total frames'].max())
  for i in range(len(all_timestamps)):
    filepath = f"/content/drive/My Drive/Colab Notebooks/data/sub001/2drt/recon/{all_timestamps['recon']}"
    recon = data_preparer.get_recon(filepath)
    vectors = data_preparer.get_pixel_data(recon, abss, all_timestamps.iloc[i])
    tensor = torch.from_numpy(vectors.copy())
    if round(all_timestamps.iloc[i]['total frames']) != highest_framenr:
      delta_frames = highest_framenr - round(all_timestamps.iloc[i]['total frames'])
      tensor = torch.cat((tensor, torch.zeros(7, 20, delta_frames)), dim=2)
    tensor_list.append(tensor)
  final_tensor = torch.stack(tensor_list, dim=0)
  #convert the letters to numeric categories
  all_timestamps['Buchstabe'] = all_timestamps['Buchstabe'].astype('category') #convert to catergory type
  all_timestamps['letter_cat'] = all_timestamps["Buchstabe"].cat.codes # get new col with numeric category
  #define X an y
  X = final_tensor
  y = all_timestamps['letter_cat']
  return X, y


In [ ]:
#need to stack all arrays from timestamps together
#therfore torch.cat needs to be applied to all arrays that are not the largest size in dimension 2
#so we first need to find out how long the dimension 2 can be at max
#and then apply torch.cat((smallervector, torch.zeros(7,20,numbertomxlen)), dim=2)
#after that all tensors can be stacked
highest_framenr = round(timestamps['total frames'].max())
tensor_list = []
for i in range(len(timestamps)):
  vectors = data_preparer.get_pixel_data(recon, abss, timestamps.iloc[i])
  tensor = torch.from_numpy(vectors.copy())
  if round(timestamps.iloc[i]['total frames']) != highest_framenr:
    delta_frames = highest_framenr - round(timestamps.iloc[i]['total frames'])
    tensor = torch.cat((tensor, torch.zeros(7, 20, delta_frames)), dim=2)
  tensor_list.append(tensor)
final_tensor = torch.stack(tensor_list, dim=0)

In [ ]:
final_tensor.shape

torch.Size([36, 7, 20, 80])

In [ ]:
#convert it back to numpy and then to tensorflow to use it for training
np_tensor = final_tensor.numpy()
tf_tensor = tf.convert_to_tensor(np_tensor)

In [ ]:
#and the label/letter of the vectors
timestamps['Buchstabe'] = timestamps['Buchstabe'].astype('category') #convert to catergory type
timestamps['letter_cat'] = timestamps["Buchstabe"].cat.codes # get new col with numeric category

In [ ]:
#X = np_tensor.reshape((36, 7, -1))
X = np_tensor#vectors
X = np.reshape(X, (36, 80, 140))
y = timestamps['letter_cat']#letter

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
X_train.shape

(28, 80, 140)

Start the training...

Used: https://medium.com/@dclengacher/keras-lstm-recurrent-neural-networks-c1f5febde03d & https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/sentiment_analysis/Keras%20LSTM%20for%20IMDB%20Sentiment%20Classification.html

In [ ]:
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM, Bidirectional, Conv1D, concatenate, Permute, Dropout
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization

**1. LSTM Model**

In [ ]:
X = np.reshape(np_tensor, (36, 80, 140)) #due that LSTM model taskes only 3 dimension as input we reshape our data to (batchsize,timephrames, vektors*pixels)
y = timestamps['letter_cat']#letter

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
#set up the model
model = Sequential()
model.add(LSTM(10,return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=36,
          epochs=10,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=36)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.9049 - accuracy: 0.2500 - val_loss: 0.9161 - val_accuracy: 0.3750
Epoch 2/10
1/1 [==============================] - 0s 60ms/step - loss: 0.8387 - accuracy: 0.3571 - val_loss: 0.8685 - val_accuracy: 0.3750
Epoch 3/10
1/1 [==============================] - 0s 64ms/step - loss: 0.7462 - accuracy: 0.4286 - val_loss: 0.8466 - val_accuracy: 0.2500
Epoch 4/10
1/1 [==============================] - 0s 62ms/step - loss: 0.7074 - accuracy: 0.4286 - val_loss: 0.8213 - val_accuracy: 0.2500
Epoch 5/10
1/1 [==============================] - 0s 57ms/step - loss: 0.6400 - accuracy: 0.3929 - val_loss: 0.7722 - val_accuracy: 0.2500
Epoch 6/10
1/1 [==============================] - 0s 61ms/step - loss: 0.5673 - accuracy: 0.3929 - val_loss: 0.7201 - val_accuracy: 0.3750
Epoch 7/10
1/1 [==============================] - 0s 69ms/step - loss: 0.4541 - accuracy: 0.3929 - val_loss: 0.6988 - val_accuracy: 0.3750
Epoch 8/10
1/1 [=============

**2. 3D-CNN**

In [ ]:
X = np_tensor#vectors
y = timestamps['letter_cat']#letter

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
# Define the model architecture
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(7, 20, 80, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same',activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=4, validation_split=0.2)

# Evaluate the model on the test set
score = model.evaluate(X_test, y_test, batch_size=4)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/10
6/6 [==============================] - 2s 171ms/step - loss: 120.6244 - accuracy: 0.3182 - val_loss: 476.0052 - val_accuracy: 0.3333
Epoch 2/10
6/6 [==============================] - 1s 149ms/step - loss: -98.0341 - accuracy: 0.3182 - val_loss: 313.7749 - val_accuracy: 0.3333
Epoch 3/10
6/6 [==============================] - 1s 151ms/step - loss: -104.7441 - accuracy: 0.3182 - val_loss: 338.0464 - val_accuracy: 0.3333
Epoch 4/10
6/6 [==============================] - 1s 146ms/step - loss: -220.2292 - accuracy: 0.3182 - val_loss: 638.2670 - val_accuracy: 0.3333
Epoch 5/10
6/6 [==============================] - 1s 154ms/step - loss: -214.2230 - accuracy: 0.3182 - val_loss: 2279.3870 - val_accuracy: 0.3333
Epoch 6/10
6/6 [==============================] - 1s 142ms/step - loss: -1110.5559 - accuracy: 0.3182 - val_loss: 3405.1633 - val_accuracy: 0.3333
Epoch 7/10
6/6 [==============================] - 1s 151ms/step - loss: -1141.2543 - accuracy: 0.3182 - val_loss: 9065.5244 - val